In [1]:
import numpy as np
import random

In [2]:
DOMAIN = list(np.arange(1,10))
DOMAIN

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
def create_grid():
    entire_grid = []
    for i in range(9):
        sub_grid = np.random.choice(DOMAIN,9).reshape(3,3)  
#         sub_grid = np.array(random.sample(DOMAIN,9)).reshape(3,3) 
        entire_grid.append(sub_grid)

    entire_grid = np.array(entire_grid).reshape(9,9)
    return entire_grid

In [4]:
def correct_row(grid,row_no,place_zero=True):

    row = grid[row_no]
    duplicate = set()

    for index,old_num in enumerate(row):

        if(old_num not in duplicate):
            duplicate.add(old_num)
        else:
            if(place_zero):
                grid[row_no][index] = 0
            else:
                possibilities = np.array([i for i in DOMAIN if i not in duplicate])
                new_num = np.random.choice(possibilities)
                grid[row_no][index] = new_num
                duplicate.add(new_num)
               
    return grid

def correct_col(grid,col_no,place_zero=True):
    
    grid = grid.transpose()
    col = grid[col_no]
    duplicate = set()

    for index,old_num in enumerate(col):

        if(old_num not in duplicate):
            duplicate.add(old_num)
        else:
            if(place_zero):
                grid[col_no][index] = 0
            else:
                possibilities = np.array([i for i in DOMAIN if i not in duplicate])
                new_num = np.random.choice(possibilities)
                grid[col_no][index] = new_num
                duplicate.add(new_num)
               
    return grid.transpose()

def correct_sub_grid(grid,sub_grid_no,place_zero=True):
    
    row_no_start = (sub_grid_no // 3) * 3
    col_no_start = (sub_grid_no % 3) * 3
#     print(row_no_start,col_no_start)
    duplicate = set()

    for row_no in range(row_no_start,row_no_start+3):
#         print(grid[row_no])
        for col_no in range(col_no_start,col_no_start+3):
#             print(grid.transpose()[col_no])

            if(grid[row_no][col_no] not in duplicate):
                duplicate.add(grid[row_no][col_no])
            else:
                if(place_zero):
                    grid[row_no][col_no] = 0
                else:
                    possibilities = np.array([i for i in DOMAIN if i not in duplicate])
                    new_num = np.random.choice(possibilities)
                    grid[row_no][col_no] = new_num
                    duplicate.add(new_num)
    
    return grid

In [5]:
def fix_values(grid):
    
    for k in range(9):
        grid = correct_row(grid,k)
        grid = correct_col(grid,k)
        grid = correct_sub_grid(grid,k)
    return grid

In [6]:
def check_row_wise(grid):
    total_invalid_entries = []
    
    for row_no in range(len(grid)):
        row = grid[row_no]
        
        if(len(set(row)) != 9): # means that elements are not unique          
            total_invalid_entries.append(row_no)
            
    return total_invalid_entries

In [7]:
def check_col_wise(grid):
    total_invalid_entries = []

    for col_no in range(len(grid)):
        col = grid.transpose()[col_no]
        
        if(len(set(col)) != 9): # means that elements are not unique      
            total_invalid_entries.append(col_no)
    
    return total_invalid_entries

In [8]:
def check_subgrid_wise(grid):
    total_invalid_entries = []
    
    for grid_no in range(len(grid)):
        row_no_start = (grid_no // 3) * 3
        col_no_start = (grid_no % 3) * 3
        duplicate = set()
        
        for row_no in range(row_no_start,row_no_start+3):
            for col_no in range(col_no_start,col_no_start+3):
                
                if(grid[row_no][col_no] not in duplicate):
                    duplicate.add(grid[row_no][col_no])
                else:
                    total_invalid_entries.append((row_no,col_no))
                    
    return total_invalid_entries

In [9]:
def reset_fixed(grid):
    
    for row_no in range(9):
        for col_no in range(9):

            current_num = grid[row_no][col_no]
#             print(suduko)
            
            if(current_num !=0):

                if(valid(row_no,col_no,current_num,grid)): # if current num is valid then leave it as it is
                    print(f'valid entry at {row_no},{col_no}')
    #                 pass

                else:
                    row = grid[row_no]
                    col = grid.transpose()[col_no]

                    sub_grid_row_no = (row_no // 3) * 3
                    sub_grid_col_no = (col_no // 3) * 3

                    sub_grid = (grid[sub_grid_row_no:sub_grid_row_no+3, sub_grid_col_no:sub_grid_col_no+3]).flatten()

                    current = set(list(row) + list(col) + list(sub_grid)) # to remove duplicates
                    reduced_possible = [i for i in DOMAIN if i not in current]

    #                 print(f'for cell {row_no},{col_no} = {reduced_possible}')

                    if(len(reduced_possible) != 0) : # means there is some value possible for cell
                        grid[row_no][col_no] = random.choice(reduced_possible)
                    else:
                        grid[row_no][col_no] = 0

#         print(grid)
    return grid

In [10]:
def valid(row_no,col_no,new_num,grid):

    in_row = np.where(grid[row_no] ==  new_num, True, False)
    in_col = np.where(grid.transpose()[col_no] == new_num, True, False)

    row_start = (row_no // 3) * 3
    col_start = (col_no // 3) * 3
    
    in_sub_grid = np.where(grid[row_start:row_start+3, col_start:col_start+3] == new_num, True, False)
    
#     print(in_row)
#     print(in_col)
#     print(in_sub_grid)
    
#     print(suduko)
    if(np.any(in_row) or np.any(in_col) or np.any(in_sub_grid)):
        return False
    else:
        return True

In [11]:
def solve(grid):
    
    if(0 in grid):
        row_no = np.where(grid == 0)[0][0]
        col_no = np.where(grid == 0)[1][0]

#         print(grid)
#         print(row_no,col_no)

        for new_num in DOMAIN:

            if(valid(row_no,col_no,new_num,grid)):
                grid[row_no][col_no] = new_num
#                 print(grid)

                if(solve(grid)):
                    return True

                grid[row_no][col_no] = 0 # if fails

#         print(f'No value left for cell {row_no},{col_no}')
        return False
    
    else:
#         print('Solution found')
        return True

In [12]:
def print_info(grid):
    
    incorrect_rows = check_row_wise(suduko)
    incorrect_cols = check_col_wise(suduko)
    incorrect_sub_grids = check_subgrid_wise(suduko)
    
    print(f'Incorrect Rows: {incorrect_rows}')
    print(f'Incorrect Columns: {incorrect_rows}')
    print(f'Incorrect Sub-grids: {incorrect_sub_grids}',end='\n\n')

    incorrect_entries = len(incorrect_rows) + len(incorrect_cols) + len(incorrect_sub_grids)
    return incorrect_entries

In [13]:
def Solve_suduko_puzzle(suduko):
    
    suduko = fix_values(suduko)
    print(f'Grid with fixed values: ')
    print(suduko, end='\n\n')

    incorrect_entries = print_info(suduko)
    print(f'Total Incorrect entries are: {incorrect_entries}',end='\n\n')

    while(True):
        
        if(solve(suduko)):
            print('Solution found')
            print(suduko, end='\n\n')
            
            current_incorrect = print_info(suduko)
            corrected_entries = incorrect_entries - current_incorrect
            print(f'Total Correct entries are: {corrected_entries}',end='\n\n')
            
            
            return suduko,corrected_entries
        
        suduko = reset_fixed(suduko)
        if(81 - len(np.nonzero(suduko)[0]) <= 17):
            print('Not enough clues')
            break

# MAIN

In [14]:
suduko = create_grid()
print(f'Random grid: ')
print(suduko,end='\n\n')

suduko_puzzle,corrected_entries = Solve_suduko_puzzle(suduko)

Random grid: 
[[8 6 5 3 2 1 4 2 6]
 [4 7 8 9 6 1 5 6 1]
 [7 7 7 8 6 8 2 5 2]
 [8 5 5 7 6 2 2 5 8]
 [3 6 7 3 2 8 8 2 3]
 [2 6 7 9 4 2 2 8 3]
 [7 1 8 7 7 8 1 5 1]
 [8 4 8 1 6 3 3 4 2]
 [9 8 3 3 5 6 5 1 2]]

Grid with fixed values: 
[[8 6 5 3 2 1 4 0 0]
 [4 7 0 9 6 0 5 0 0]
 [0 0 0 8 0 0 2 0 0]
 [0 5 0 7 0 2 0 0 8]
 [3 0 7 0 0 8 0 0 0]
 [2 0 0 0 4 0 0 0 3]
 [0 1 8 0 7 0 0 5 0]
 [0 4 0 1 0 3 0 0 2]
 [9 0 3 0 5 6 0 1 0]]

Incorrect Rows: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Incorrect Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Incorrect Sub-grids: [(2, 0), (2, 1), (2, 2), (2, 4), (2, 5), (0, 8), (1, 7), (1, 8), (2, 7), (2, 8), (3, 2), (4, 1), (5, 1), (5, 2), (4, 3), (4, 4), (5, 3), (5, 5), (3, 7), (4, 6), (4, 7), (4, 8), (5, 6), (5, 7), (7, 0), (7, 2), (8, 1), (6, 5), (7, 4), (8, 3), (6, 8), (7, 6), (7, 7), (8, 6), (8, 8)]

Total Incorrect entries are: 53

Solution found
[[8 1 4 2 7 9 6 3 5]
 [6 2 5 1 3 4 8 9 7]
 [3 7 9 8 5 6 4 1 2]
 [2 4 8 6 1 7 9 5 3]
 [9 5 3 4 8 2 1 7 6]
 [7 6 1 5 9 3 2 4 8]
 [5 9 2 3 6

In [15]:
print(suduko_puzzle)
print(corrected_entries)

[[8 1 4 2 7 9 6 3 5]
 [6 2 5 1 3 4 8 9 7]
 [3 7 9 8 5 6 4 1 2]
 [2 4 8 6 1 7 9 5 3]
 [9 5 3 4 8 2 1 7 6]
 [7 6 1 5 9 3 2 4 8]
 [5 9 2 3 6 1 7 8 4]
 [4 8 7 9 2 5 3 6 1]
 [1 3 6 7 4 8 5 2 9]]
53


In [16]:
print(check_row_wise(suduko_puzzle))
print(check_col_wise(suduko_puzzle))
print(check_subgrid_wise(suduko_puzzle))

[]
[]
[]
